# Create a RAG with Responses API

TO EXECUTE, YOU NEED TO FILL INPUT DATA

* [OpenAI Responses API](https://platform.openai.com/docs/api-reference/responses)
* [Azure OpenAI Responses API](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-key)
* [Vertex AI RAG Engine overview](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-engine/rag-overview?_gl=1*1gh4852*_up*MQ..&gclid=CjwKCAjw2vTFBhAuEiwAFaScwheDO8BoxS9t35aQYzYpVChMuVZr8xP7S7bEUNvDPrXD7bD8gZOPKxoC_Z0QAvD_BwE&gclsrc=aw.ds)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from src import utils, conf

# Params

In [5]:
settings = conf.load(file="settings.yaml")
INDEX = settings.vdb_index
LLM = settings.llm_workhorse
RETRIEVE_K = 3

# Environment Variables

In [6]:
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Clients

In [7]:
client_openai = OpenAI(
    api_key=OPENAI_API_KEY
)

# Upload Data

In [8]:
# Identify every .pdf from data/raw/
pdf_files = [f for f in utils.path_data_raw.glob("*.pdf")]
print(f"{pdf_files=}")

pdf_files=[WindowsPath('c:/Users/manuelalberto.romero/Documents/repos/dslabs/dslab-rag-e2e/data/raw/3I_ATLAS, NAUKAS.pdf'), WindowsPath('c:/Users/manuelalberto.romero/Documents/repos/dslabs/dslab-rag-e2e/data/raw/Divulgacion Planetaria Althera.pdf'), WindowsPath('c:/Users/manuelalberto.romero/Documents/repos/dslabs/dslab-rag-e2e/data/raw/OMUAMUA. NAUKAS.pdf')]


In [ ]:
file_path = utils.path_data_raw / "Divulgacion Planetaria Althera.pdf"
id_file = 'file-EyjaMAEtuh7L6TyJMHXgK3'  # add file id once created from cell output print on the first run

lst_oai_files = [x.id for x in client_openai.files.list()]

if id_file not in lst_oai_files:
    response_file = client_openai.files.create(
        file=open(file_path, 'rb'),
        purpose="assistants",
        expires_after={
            "anchor": "created_at",
            "seconds": 2592000   # 30d
        })
    print(f"Added file: {response_file.id=}")
else:
    print(f"File: {id_file} alredy added")

Added file: response_file.id='file-EyjaMAEtuh7L6TyJMHXgK3'


# Index

In [11]:
id_vdb = "vs_68c33a972b5081918fe563b20b860e31"   # add vector store id once created from cell output print on the first run

lst_oai_vdb =[x.id for x in client_openai.vector_stores.list()]

if id_vdb not in lst_oai_vdb:
    response_attach = client_openai.vector_stores.create(
        name=INDEX,
        file_ids=[response_file.id],
        expires_after={
            "anchor": "last_active_at",
            "days": 30
            }
    )
    print(f"Created VDB: {response_attach.id=}")
else:
    print(f"VDB: {id_vdb} alredy exists")



VDB: vs_68c33a972b5081918fe563b20b860e31 alredy exists


In [13]:
id_vdb

'vs_68c33a972b5081918fe563b20b860e31'

In [14]:
id_file

'file-F5KwBtf7a6juA5LDgoQ1Nf'

In [16]:
# max_chunk_size_tokens of 800 and chunk_overlap_tokens of 400.
vector_store_files = client_openai.vector_stores.files.list(
  vector_store_id=id_vdb
)

lst_indexed_files = [x.id for x in vector_store_files.data]
if id_file not in lst_indexed_files:
    client_openai.vector_stores.files.create(
        vector_store_id=id_vdb,
        file_id=id_file
    )
    print(f"Indexed file: {id_file=} in VDB: {id_vdb=}")


# Query

In [17]:
query = "¿A qué distancia de la Tierra está el sistema Althéra?"
res_search = client_openai.vector_stores.search(
    vector_store_id=id_vdb,
    query=query,
    max_num_results=3
)

In [18]:
for result in res_search.data:
    print(str(len(result.content[0].text)) + ' of character of content from ' + result.filename + ' with a relevant score of ' + str(result.score))

2550 of character of content from Divulgacion Planetaria Althera.pdf with a relevant score of 0.9878389379686081
2292 of character of content from Divulgacion Planetaria Althera.pdf with a relevant score of 0.8624259655500602
2602 of character of content from Divulgacion Planetaria Althera.pdf with a relevant score of 0.8129702299754368


# RAG

In [19]:
query = "¿A qué distancia de la Tierra está el sistema Althéra?"

res_rag = client_openai.responses.create(
    input= query,
    model=LLM,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [id_vdb],
    }]
)

# Extract annotations from the response
annotations = res_rag.output[1].content[0].annotations
    
# Get top-k retrieved filenames
retrieved_files = set([result.filename for result in annotations])

print(f'Files used: {retrieved_files}')
print('Response:')
print(res_rag.output[1].content[0].text) # 0 being the filesearch call

Files used: {'Divulgacion Planetaria Althera.pdf'}
Response:
El sistema binario Althéra está situado a una distancia de aproximadamente 42,7 años luz de la Tierra. Se encuentra en el brazo de Orión de la Vía Láctea, próximo a la nebulosa Barnard's Loop en la constelación de Orión. Esta relativa cercanía permite realizar observaciones detalladas con telescopios avanzados y ha hecho posible estudiar con detalle sus planetas y características astronómicas.


In [20]:
annotations[0]

AnnotationFileCitation(file_id='file-EyjaMAEtuh7L6TyJMHXgK3', filename='Divulgacion Planetaria Althera.pdf', index=395, type='file_citation')

In [21]:
res_rag.usage.total_tokens

16298

In [22]:
res_rag.output[0]  # ResponseFileSearchToolCall

ResponseFileSearchToolCall(id='fs_68c33b4bb2c48195bee704818b9c6351095f36b665015d33', queries=['distancia del sistema Althéra a la Tierra', 'Althéra distancia Tierra', '¿A qué distancia de la Tierra está el sistema Althéra?'], status='completed', type='file_search_call', results=None)

In [23]:
res_rag.output[1]  # ResponseOutputMessage

ResponseOutputMessage(id='msg_68c33b4dcfd48195a505febd8fca6a01095f36b665015d33', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-EyjaMAEtuh7L6TyJMHXgK3', filename='Divulgacion Planetaria Althera.pdf', index=395, type='file_citation')], text="El sistema binario Althéra está situado a una distancia de aproximadamente 42,7 años luz de la Tierra. Se encuentra en el brazo de Orión de la Vía Láctea, próximo a la nebulosa Barnard's Loop en la constelación de Orión. Esta relativa cercanía permite realizar observaciones detalladas con telescopios avanzados y ha hecho posible estudiar con detalle sus planetas y características astronómicas.", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')